In [ ]:
#Import libraries
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#  Load embedding models
ann_embed_model  = load_model("/content/ann_embedding_model.h5", compile=False)
cnn_embed_model  = load_model("/content/cnn_embedding_model.h5", compile=False)
lstm_embed_model = load_model("/content/fusion_ready_lstm_embedding_model.h5", compile=False)

# 3️⃣ Prepare tabular (ANN) data
df = pd.read_csv("/content/Feature_engineering.csv")
X_tab = df.drop("HeartDiseaseorAttack", axis=1)
y     = df["HeartDiseaseorAttack"]

X_train_tab, X_val_tab, y_train, y_val = train_test_split(
    X_tab, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_tab_scaled = scaler.fit_transform(X_train_tab)
X_val_tab_scaled   = scaler.transform(X_val_tab)

# Prepare CNN (image) data
img_size = (224, 224)
batch_size = 1

datagen = ImageDataGenerator(rescale=1./255)

train_img_gen = datagen.flow_from_directory(
    "/content/heart_disease/chest_xray/chest_xray/train",
    target_size=img_size,
    class_mode='binary',
    shuffle=False,
    batch_size=1
)

val_img_gen = datagen.flow_from_directory(
    "/content/heart_disease/chest_xray/chest_xray/val",
    target_size=img_size,
    class_mode='binary',
    shuffle=False,
    batch_size=1
)


#  Prepare LSTM (ECG) data
ecg_train = pd.read_csv('/content/ECG/mitbih_train.csv', header=None)
ecg_test  = pd.read_csv('/content/ECG/mitbih_test.csv', header=None)

X_ecg = ecg_train.iloc[:, :-1].values / ecg_train.iloc[:, :-1].values.max()
y_ecg = np.where(ecg_train.iloc[:, -1].values == 0, 0, 1)
X_ecg = X_ecg[:, :, None]

X_train_ecg, X_val_ecg, _, _ = train_test_split(
    X_ecg, y_ecg, test_size=0.2, random_state=42, stratify=y_ecg
)

# Match dataset lengths
min_len_train = min(len(X_train_tab_scaled), len(train_img_gen), len(X_train_ecg))
min_len_val   = min(len(X_val_tab_scaled), len(val_img_gen), len(X_val_ecg))

X_tab_train_fusion = X_train_tab_scaled[:min_len_train]
X_tab_val_fusion   = X_val_tab_scaled[:min_len_val]

X_img_train_fusion = np.array([train_img_gen[i][0][0] for i in range(min_len_train)])
X_img_val_fusion   = np.array([val_img_gen[i][0][0] for i in range(min_len_val)])

X_ecg_train_fusion = X_train_ecg[:min_len_train]
X_ecg_val_fusion   = X_val_ecg[:min_len_val]

y_train_fusion = y_train.values[:min_len_train]
y_val_fusion   = y_val.values[:min_len_val]

# Extract embeddings
ann_embeddings_train  = ann_embed_model.predict(X_tab_train_fusion)
ann_embeddings_val    = ann_embed_model.predict(X_tab_val_fusion)

cnn_embeddings_train  = cnn_embed_model.predict(X_img_train_fusion)
cnn_embeddings_val    = cnn_embed_model.predict(X_img_val_fusion)

lstm_embeddings_train = lstm_embed_model.predict(X_ecg_train_fusion)
lstm_embeddings_val   = lstm_embed_model.predict(X_ecg_val_fusion)

# 8️⃣ Build Fusion Model
tab_in  = Input(shape=ann_embeddings_train.shape[1:], name="tab_input_fusion")
cnn_in  = Input(shape=cnn_embeddings_train.shape[1:], name="cnn_input_fusion")
lstm_in = Input(shape=lstm_embeddings_train.shape[1:], name="lstm_input_fusion")

combined = Concatenate(name="fusion_concat")([tab_in, cnn_in, lstm_in])

x = Dense(256, activation='relu', name="fusion_dense_1")(combined)
x = Dropout(0.3, name="fusion_dropout_1")(x)
x = Dense(128, activation='relu', name="fusion_dense_2")(x)
x = Dropout(0.2, name="fusion_dropout_2")(x)
output = Dense(1, activation='sigmoid', name="fusion_output")(x)

fusion_model = Model(inputs=[tab_in, cnn_in, lstm_in], outputs=output, name="Fusion_Model")
fusion_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
fusion_model.summary()

# Train Fusion Model
fusion_model.fit(
    [ann_embeddings_train, cnn_embeddings_train, lstm_embeddings_train],
    y_train_fusion,
    validation_data=([ann_embeddings_val, cnn_embeddings_val, lstm_embeddings_val], y_val_fusion),
    epochs=10,
    batch_size=32
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
163/163 ━━━━━━━━━━━━━━━━━━━━ 207s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
163/163 ━━━━━━━━━━━━━━━━━━━━ 11s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step


Model: "Fusion_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ tab_input_fusion    │ (None, 32)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_input_fusion    │ (None, 1280)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_input_fusion   │ (None, 64)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_concat       │ (None, 1376)      │          0 │ tab_input_fusion… │
│ (Concatenate)       │                   │            │ cnn_input_fusion… │
│                     │                   │            │ lstm_input_fusio… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dense_1      │ (None, 256)       │    352,512 │ fusion_concat[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dropout_1    │ (None, 256)       │          0 │ fusion_dense_1[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dense_2      │ (None, 128)       │     32,896 │ fusion_dropout_1… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_dropout_2    │ (None, 128)       │          0 │ fusion_dense_2[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fusion_output       │ (None, 1)         │        129 │ fusion_dropout_2… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 385,537 (1.47 MB)

 Trainable params: 385,537 (1.47 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.8881 - loss: 0.3240 - val_accuracy: 0.8750 - val_loss: 0.3366
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9029 - loss: 0.2607 - val_accuracy: 0.8750 - val_loss: 0.2678
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9022 - loss: 0.2533 - val_accuracy: 0.8750 - val_loss: 0.2592
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9092 - loss: 0.2313 - val_accuracy: 0.8750 - val_loss: 0.2567
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9077 - loss: 0.2469 - val_accuracy: 0.8750 - val_loss: 0.2480
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8970 - loss: 0.2500 - val_accuracy: 0.8750 - val_loss: 0.2514
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9074 - loss: 0.2380 - val_accuracy: 0.8750 - val_loss: 0.2112
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9034 - loss: 0.2525 - val_accuracy

In [ ]:
fusion_model.save("heart_disease_fusion_model.h5")


In [ ]:
#Import libraries
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import StandardScaler

#Load models
ann_embed_model  = load_model("/content/ann_embedding_model.h5", compile=False)
cnn_embed_model  = load_model("/content/cnn_embedding_model.h5", compile=False)
lstm_embed_model = load_model("/content/fusion_ready_lstm_embedding_model.h5", compile=False)
fusion_model    = load_model("/content/heart_disease_fusion_model.h5", compile=False)

#Prepare scaler for tabular data
df = pd.read_csv("/content/Feature_engineering.csv")
X_tab = df.drop("HeartDiseaseorAttack", axis=1)

scaler = StandardScaler()
scaler.fit(X_tab)

#  Function to predict for one patient
def predict_patient(tabular_data, ecg_data, img_path):
    """
    tabular_data: pd.DataFrame or np.array of shape (1, num_features)
    ecg_data: np.array of shape (1, timesteps)
    img_path: string path to X-ray image
    """

    # ANN embedding
    tab_scaled = scaler.transform(tabular_data)
    ann_emb = ann_embed_model.predict(tab_scaled)

    # LSTM embedding
    ecg_reshaped = ecg_data[:, :, None]
    lstm_emb = lstm_embed_model.predict(ecg_reshaped)

    # CNN embedding
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)
    cnn_emb = cnn_embed_model.predict(img_array)

    #Fusion model prediction
    pred = fusion_model.predict([ann_emb, cnn_emb, lstm_emb])
    pred_class = (pred > 0.5).astype(int)

    return pred[0][0], pred_class[0][0]

# Example usage
tabular_example = X_tab.iloc[0:1]
ecg_example     = np.array([[0]*187])
xray_example    = "/content/heart_disease/chest_xray/test/NORMAL/IM-0001-0001.jpeg"
probability, prediction = predict_patient(tabular_example, ecg_example, xray_example)
print(f"✅ Heart Disease Probability: {probability:.4f}")
print(f"✅ Predicted Class (0=No, 1=Yes): {prediction}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
✅ Heart Disease Probability: 0.3914
✅ Predicted Class (0=No, 1=Yes): 0


In [ ]:
# After training fusion model
fusion_model.save("/content/fusion_model.h5")

from tensorflow.keras.models import load_model
fusion_model = load_model("/content/fusion_model.h5", compile=False)
